In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()], )

In [16]:
chef_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",
         "너는 한국 최고의 요리사이고, 세계의 요리를 한국 음식 스타일로 만들어 내는 방법을 알고 있어. 또한 그 레시피를 아주 쉽게 설명할 수 있어"
         ),
        ("human", "{cuisine} 음식을 만들어 보고 싶어")
    ]
)

chef_chain = chef_prompt | chat
chef_chain.invoke({"cuisine": "이탈리아"})

AIMessage(content='그럼 한국 음식 스타일로 이탈리아 요리를 만들어 볼까요? 파스타 카본라라는 이탈리아 요리를 한국 음식 스타일로 변형해 보겠습니다. 준비물은 스팸, 양파, 마늘, 계란, 파슬리, 스파게티면, 소금, 후추, 파마산 치즈, 올리브 오일이 필요합니다.\n\n1. 먼저 양파와 마늘을 다져주세요.\n2. 팬에 올리브 오일을 두르고 양파와 마늘을 볶아주세요.\n3. 스팸을 다져서 팬에 넣고 볶아주세요.\n4. 스팸이 익으면 소금과 후추로 간을 해주세요.\n5. 다른 팬에 계란을 풀어 스크램블을 만들어주세요.\n6. 스파게티면을 삶아주세요.\n7. 삶은 스파게티면을 팬에 넣고 볶아주세요.\n8. 볶은 스파게티면에 스팸과 양파를 넣고 섞어주세요.\n9. 마지막으로 스크램블된 계란과 다진 파슬리를 넣고 섞어주세요.\n10. 그릇에盛り付け하여 파마산 치즈를 뿌려 완성하세요.\n\n이렇게 간단하게 한국 음식 스타일의 파스타 카본라를 만들 수 있어요. 맛있게 즐기세요!')

In [25]:
veg_chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "너는 채식주의자이고, 채식을 즐기는 사람들에게 쉽게 레시피를 제공해. 특히 주어진 {recipe}에서 고기를 대체할 수 있는 레시피를 만들고 맛 또한 비슷하게 만들 수 있어. 하지만 방법이 없다면 모른다고 말해"
        ),
        ("human", "{recipe}")
    ]
)
veg_chef_chain = veg_chef_prompt | chat

veg_recipe_chain = {"recipe": chef_chain} | veg_chef_chain
veg_recipe_chain.invoke({"cuisine": "중국"})


AIMessage(content='고기를 대체할 수 있는 레시피를 만들어 볼까요? \n\n한국식 짜장면 레시피를 채식주의자용으로 변형해보겠습니다.\n\n채식주의자용 한국식 짜장면 레시피:\n재료:\n- 중국당면 200g\n- 버섯(새송이, 표고버섯 등) 200g\n- 양파 1개\n- 당근 1/2개\n- 양배추 1/4개\n- 대파 1대\n- 식용유 2큰술\n- 짜장소스 4큰술\n- 물 1컵\n- 소금, 후추 약간\n\n1. 먼저 중국당면을 물에 불려주세요. 불린 후 물기를 제거해주세요.\n2. 버섯을 채 썰어 준비해주세요.\n3. 양파, 당근, 양배추를 채썰어 준비해주세요.\n4. 팬에 식용유를 두르고 버섯을 볶아주세요.\n5. 볶은 버섯에 양파, 당근, 양배추를 넣고 함께 볶아주세요.\n6. 볶은 채소에 짜장소스를 넣고 볶아주세요.\n7. 물을 넣고 끓여주세요.\n8. 끓인 물에 불린 중국당면을 넣고 끓여주세요.\n9. 대파를 썰어 마무리해주세요.\n\n이렇게 간단하게 채식주의자용 한국식 짜장면을 만들 수 있어요. 고기 대신에 버섯을 사용하여 비슷한 맛을 내보세요. 맛있게 즐기세요!')